In [1]:
def My_ConvLSTM_Model(frames, channels, pixels_x, pixels_y, categories):
  
    trailer_input  = Input(shape=(frames, channels, pixels_x, pixels_y)
                    , name='trailer_input')
    
    first_ConvLSTM = ConvLSTM2D(filters=20, kernel_size=(3, 3)
                       , data_format='channels_first'
                       , recurrent_activation='hard_sigmoid'
                       , activation='tanh'
                       , padding='same', return_sequences=True)(trailer_input)
    first_BatchNormalization = BatchNormalization()(first_ConvLSTM)
    first_Pooling = MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_first')(first_BatchNormalization)
    
    second_ConvLSTM = ConvLSTM2D(filters=10, kernel_size=(3, 3)
                        , data_format='channels_first'
                        , padding='same', return_sequences=True)(first_Pooling)
    second_BatchNormalization = BatchNormalization()(second_ConvLSTM)
    second_Pooling = MaxPooling3D(pool_size=(1, 3, 3), padding='same', data_format='channels_first')(second_BatchNormalization)
    
    outputs = [branch(second_Pooling, 'cat_{}'.format(category)) for category in categories]
    
    seq = Model(inputs=trailer_input, outputs=outputs, name='Model ')
    
    return seq

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pylab as plt

In [4]:
seq = keras.Sequential(
    [
        keras.Input(
            shape=(None, 227, 227, 1)  
            # (time, rows, cols, channels)
            # 이 중, time을 지정하지 가변으로 두고, (None, 40,40,1) shape의 프레임을 입력받습니다.
        ),  # 40x40에 1채널을 가지는 프레임입니다.
        layers.ConvLSTM2D(
            filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
        layers.BatchNormalization(),
        layers.Conv3D(
            filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
        ),
    ]
)
seq.compile(loss="binary_crossentropy", optimizer="adadelta")

In [5]:
seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, None, 227, 227, 4  59200     
                             0)                                  
                                                                 
 batch_normalization (BatchN  (None, None, 227, 227, 4  160      
 ormalization)               0)                                  
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, None, 227, 227, 4  115360    
                             0)                                  
                                                                 
 batch_normalization_1 (Batc  (None, None, 227, 227, 4  160      
 hNormalization)             0)                                  
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, None, 227, 227, 4  1